In [105]:
import pandas as pd
import numpy as np
import threading
import time

from jedi.inference import InferenceState
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import gc
from typing import Dict, List, Tuple, Optional, Any
from queue import Queue
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os
import datetime

warnings.filterwarnings('ignore')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
File_Path = '/Users/shreyasravi/PycharmProjects/Embedded-Systems/London_Weather.csv'

A thread-safe class for preprocessing weather data, implementing feature selection, and preparing data chunks for LSTM training and inference processes. This class acts as a producer in a producer-consumer pattern, sending data to both training and inference threads.

Semaphore is like a counter which is used to control the number of threads that can access a shared resource at the same time. It is a signaling mechanism like traffic light block threads when resources are not available and allow threads when the resources are available.

In the LSTM, there are three key gates:

1. The Forget Gate manages what should be forgotten.
2. The Input Gate manages what should be kept, and
3. The Output Gate manages what information is stored in the carry and hidden states.

Each of these gates comprise of their own neural network layer that handles the mathematics needed to retain the relevant data we need to store in both the short-term memory and long-term memory.

This class acts as a consumer in the producer-consumer pattern,
consuming data from the DataPreprocessing class and training an LSTM model.

Thread-safe class for inferring using a trained LSTM model for weather forecasting.

This class acts as a consumer in a producer-consumer pattern, consuming test data
from the preprocessing thread and generating predictions using the trained model.

In [106]:
class Preprocessing:
    def __init__(self, file_path):
        self.file_path = file_path
        self.data = None
        self.scaler = None
        self.normalized_data = None
        self.train_size = None
        self.X_train = None
        self.y_train = None
        self.X_test = None
        self.y_test = None
        self.T = 20  # Number of timesteps to look while predicting

    def load_data(self):
        self.data = pd.read_csv(self.file_path)
        return self.data

    def clean_data(self):
        # Convert date to datetime format
        self.data['date'] = pd.to_datetime(self.data['date'], format='%Y%m%d')

        # Fill missing values in snow_depth with 0
        self.data['snow_depth'].fillna(0, inplace=True)

        # Drop rows with any NaN values
        self.data.dropna(inplace=True)

        return self.data

    def prepare_data_for_model(self, train_ratio=0.8, target_column='mean_temp'):
        # Separate features and target
        input_data = self.data.drop(['date'], axis=1)
        targets = self.data[target_column].values

        # Define dimensions
        D = input_data.shape[1]  # Number of features
        N = len(input_data) - self.T

        # Calculate train size
        self.train_size = int(len(input_data) * train_ratio)

        # Normalize input data
        self.scaler = StandardScaler()
        self.scaler.fit(input_data[:self.train_size + self.T - 1])
        self.normalized_data = self.scaler.transform(input_data)

        # Prepare X_train and y_train
        self.X_train = np.zeros((self.train_size, self.T, D))
        self.y_train = np.zeros((self.train_size, 1))

        for t in range(self.train_size):
            self.X_train[t, :, :] = self.normalized_data[t:t+self.T]
            self.y_train[t] = targets[t+self.T]

        # Prepare X_test and y_test
        self.X_test = np.zeros((N - self.train_size, self.T, D))
        self.y_test = np.zeros((N - self.train_size, 1))

        for i in range(N - self.train_size):
            t = i + self.train_size
            self.X_test[i, :, :] = self.normalized_data[t:t+self.T]
            self.y_test[i] = targets[t+self.T]

        # Convert to PyTorch tensors
        self.X_train = torch.from_numpy(self.X_train.astype(np.float32))
        self.y_train = torch.from_numpy(self.y_train.astype(np.float32))
        self.X_test = torch.from_numpy(self.X_test.astype(np.float32))
        self.y_test = torch.from_numpy(self.y_test.astype(np.float32))

        return self.X_train, self.y_train, self.X_test, self.y_test

    def get_train_test_data(self):
        return self.X_train, self.y_train, self.X_test, self.y_test

    def get_original_data(self):
        return self.data

    def get_scaler(self):
        return self.scaler


In [107]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.M = hidden_dim
        self.L = layer_dim

        self.rnn = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=layer_dim,
            batch_first=True
        )
        # batch_first to have (batch_dim, seq_dim, feature_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, X):
        # Set device - ideally this would be a class property set during initialization
        device = X.device

        # Initial hidden state and cell state
        h0 = torch.zeros(self.L, X.size(0), self.M).to(device)
        c0 = torch.zeros(self.L, X.size(0), self.M).to(device)

        out, (hn, cn) = self.rnn(X, (h0.detach(), c0.detach()))

        # h(T) at the final time step
        out = self.fc(out[:, -1, :])
        return out

In [108]:
class Training:
    def __init__(self):
        self.model = None
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.train_losses = None
        self.test_losses = None

    def create_model(self, input_dim, hidden_dim=512, layer_dim=2, output_dim=1):
        self.model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
        self.model.to(self.device)
        return self.model

    def train_model(self, X_train, y_train, X_test, y_test, learning_rate=0.01, epochs=200):
        # Move data to device
        X_train, y_train = X_train.to(self.device), y_train.to(self.device)
        X_test, y_test = X_test.to(self.device), y_test.to(self.device)

        # Loss and optimizer
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(
            self.model.parameters(),
            lr=learning_rate,
            momentum=0.9,
            weight_decay=1e-4
        )

        self.train_losses = np.zeros(epochs)
        self.test_losses = np.zeros(epochs)

        for epoch in range(epochs):
            optimizer.zero_grad()

            # Forward pass
            outputs = self.model(X_train)
            loss = criterion(outputs, y_train)

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Train loss
            self.train_losses[epoch] = loss.item()

            # Test loss
            test_outputs = self.model(X_test)
            test_loss = criterion(test_outputs, y_test)
            self.test_losses[epoch] = test_loss.item()

        return self.train_losses, self.test_losses

    def load_model(self, model_path=None, weights_path=None):
        if model_path is not None:
            self.model = torch.load(model_path, map_location=self.device)
        elif weights_path is not None and self.model is not None:
            self.model.load_state_dict(torch.load(weights_path, map_location=self.device))

        return self.model

    def get_model(self):
        return self.model

    def get_losses(self):
        return self.train_losses, self.test_losses

In [109]:
class Inference:
    def __init__(self, model=None, device=None):
        self.model = model
        self.device = device if device is not None else torch.device(
            "cuda:0" if torch.cuda.is_available() else "cpu"
        )
        self.predictions = None

    def set_model(self, model):
        self.model = model
        self.model.to(self.device)

    def predict(self, X_test):
        # Move data to device
        X_test = X_test.to(self.device)

        # Set model to evaluation mode
        self.model.eval()

        # Generate predictions
        self.predictions = []
        with torch.no_grad():
            for i in range(len(X_test)):
                input_ = X_test[i].reshape(1, X_test.shape[1], X_test.shape[2])
                p = self.model(input_)[0, 0].item()
                self.predictions.append(p)
        return self.predictions

    def get_predictions(self):
        return self.predictions

In [110]:
class Analysis:
    def __init__(self):
        self.true_values = None
        self.predictions = None
        self.metrics = {}

    def set_data(self, true_values, predictions):
        if isinstance(true_values, torch.Tensor):
            self.true_values = true_values.cpu().detach().numpy()
        else:
            self.true_values = true_values

        self.predictions = predictions

    def calculate_metrics(self):
        if self.true_values is None or self.predictions is None:
            raise ValueError("Data not set. Call set_data first.")

        # Calculate MAE
        self.metrics['MAE'] = mean_absolute_error(self.true_values, self.predictions)

        # Calculate MSE
        self.metrics['MSE'] = mean_squared_error(self.true_values, self.predictions)

        # Calculate RMSE
        self.metrics['RMSE'] = np.sqrt(self.metrics['MSE'])

        print(f"Metrics calculated - MAE: {self.metrics['MAE']:.3f}, MSE: {self.metrics['MSE']:.3f}, RMSE: {self.metrics['RMSE']:.3f}")
        return self.metrics

    def analyze_error_distribution(self):
        # Calculate errors
        errors = self.true_values.flatten() - np.array(self.predictions)

        # Basic statistics of errors
        error_stats = {
            'mean': np.mean(errors),
            'std': np.std(errors),
            'min': np.min(errors),
            'max': np.max(errors)
        }

        print(f"Error statistics - Mean: {error_stats['mean']:.3f}, Std: {error_stats['std']:.3f}, Min: {error_stats['min']:.3f}, Max: {error_stats['max']:.3f}")
        return errors, error_stats

    def get_metrics(self):
        return self.metrics

In [111]:
class Display:
    def __init__(self):
        self.data = None
        self.predictions = None
        self.dates = None

    def set_data(self, original_data, predictions, start_idx=None):
        self.data = original_data
        self.predictions = predictions

        # Prepare the plot DataFrame
        plot_len = len(predictions)
        if start_idx is None:
            start_idx = -plot_len

        self.plot_df = original_data[['date', 'mean_temp']].copy(deep=True)
        self.plot_df = self.plot_df.iloc[start_idx:]
        self.plot_df['prediction'] = predictions
        self.plot_df.set_index('date', inplace=True)

    def plot_results(self, title=None, figsize=(20, 10)):

        plt.figure(figsize=figsize)
        plt.plot(self.plot_df['mean_temp'], label='Actual Temperature', linewidth=1)
        plt.plot(self.plot_df['prediction'], label='Predicted Temperature', linewidth=1)
        plt.xlabel('Date')
        plt.ylabel('Temperature (°C)')
        plt.legend(loc='lower right')

        if title:
            plt.title(title)

        plt.tight_layout()
        plt.show()

    def plot_by_year(self, figsize=(20, 10)):
        # Group data by year
        plot_df_by_years = []
        for y in self.plot_df.index.year.unique():
            plot_df_by_years.append((y, self.plot_df.loc[self.plot_df.index.year == y]))

        # Plot each year separately
        for year, year_df in plot_df_by_years:
            plt.figure(figsize=figsize)
            plt.plot(year_df['mean_temp'], label='Actual Temperature', linewidth=1)
            plt.plot(year_df['prediction'], label='Predicted Temperature', linewidth=1)
            plt.xlabel('Date')
            plt.ylabel('Temperature (°C)')
            plt.legend(loc='lower right')
            plt.title(f'Temperature in {year}')
            plt.tight_layout()
            plt.show()

    def plot_error_histogram(self, errors, bins=25, figsize=(12, 8)):
        plt.figure(figsize=figsize)
        plt.hist(errors, bins=bins)
        plt.xlabel('Temperature Difference (Actual - Predicted)')
        plt.ylabel('Count')
        plt.title('Distribution of Prediction Errors')
        plt.tight_layout()
        plt.show()

    def plot_training_history(self, train_losses, test_losses, figsize=(12, 8)):
        plt.figure(figsize=figsize)
        plt.plot(train_losses, label='Train Loss')
        plt.plot(test_losses, label='Test Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.title('Training History')
        plt.tight_layout()
        plt.show()

In [112]:
# Constants and configurations
FILE_PATH = 'London_Weather.csv'  # Update with your file path
CAPACITY = 15  # Size of data buffer
DATALOADER_LOAD_THRESHOLD = 1  # How many data points to load at once
TRAINER_READ_THRESHOLD = 3  # How many data points trainer reads at once
INFERENCE_READ_THRESHOLD = 1  # How many data points inference reads at once

# Wait times to simulate processing time
DATALOADER_WAIT_TIME = 1  # seconds
TRAINER_WAIT_TIME = 5  # seconds
INFERENCE_WAIT_TIME = 2  # seconds

# Training parameters
EPOCHS = 70
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Create shared data buffer for holding processed batches
data_buffer = [None for _ in range(CAPACITY)]
main_start_time = time.time()
preprocess_start_time = time.time()

# Initialize the preprocessing class
preprocessor = Preprocessing(FILE_PATH)
preprocessor.load_data()
preprocessor.clean_data()

# Prepare data for model
X_train, y_train, X_test, y_test = preprocessor.prepare_data_for_model(target_column='mean_temp')

# Fill the buffer with batches of data
datapoints_loaded = 0
in_index = 0

# We'll create mini-batches from the training data to fill the buffer
batch_size = max(1, len(X_train) // CAPACITY)  # Ensure at least 1 example per batch

while datapoints_loaded < CAPACITY:
    # Create a batch of data
    start_idx = datapoints_loaded * batch_size
    end_idx = min(start_idx + batch_size, len(X_train))

    if start_idx >= len(X_train):
        break

    batch_X = X_train[start_idx:end_idx]
    batch_y = y_train[start_idx:end_idx]

    # Store batch in buffer
    data_buffer[in_index] = (batch_X, batch_y)

    print(f"[DATA LOADER] Added batch {datapoints_loaded+1} to buffer position {in_index+1}, size: {len(batch_X)}")

    # Simulate processing time
    time.sleep(DATALOADER_WAIT_TIME)

    # Update indices and counters
    in_index = (in_index + DATALOADER_LOAD_THRESHOLD) % CAPACITY
    datapoints_loaded += DATALOADER_LOAD_THRESHOLD

preprocess_end_time = time.time()
print(f"[DATA LOADER] Completed in {preprocess_end_time - preprocess_start_time:.2f} seconds")


training_start_time = time.time()

# Initialize trainer and create model
trainer = Training()
input_dim = X_train.shape[2]  # Number of features
model = trainer.create_model(input_dim=input_dim)

# Read batches from the buffer and train
read_datapoints_in_trainer = 0
out_index_trainer = 0
train_losses = []
test_losses = []

while read_datapoints_in_trainer < CAPACITY:
    # Check if we've reached the end of useful data
    if out_index_trainer >= datapoints_loaded:
        break

    # Get batches from buffer (up to 3 at a time)
    trainer_batches_X = []
    trainer_batches_y = []

    for i in range(out_index_trainer, min(out_index_trainer + TRAINER_READ_THRESHOLD, datapoints_loaded)):
        batch_X, batch_y = data_buffer[i]
        trainer_batches_X.append(batch_X)
        trainer_batches_y.append(batch_y)

    # Combine batches
    combined_X = torch.cat(trainer_batches_X)
    combined_y = torch.cat(trainer_batches_y)

    print(f"[TRAINER] Processing batches from positions {out_index_trainer+1} to {min(out_index_trainer+TRAINER_READ_THRESHOLD, datapoints_loaded)}")

    # Train on this combined batch for a few epochs
    mini_epochs = 5  # Train for 5 epochs on each batch
    for epoch in range(mini_epochs):
        # Move data to device
        combined_X = combined_X.to(device)
        combined_y = combined_y.to(device)
        X_test_sample = X_test[:min(len(X_test), 100)].to(device)  # Use a subset for validation
        y_test_sample = y_test[:min(len(y_test), 100)].to(device)

        # Train for one epoch
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(combined_X)
        loss = criterion(outputs, combined_y)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Compute validation loss
        with torch.no_grad():
            test_outputs = model(X_test_sample)
            test_loss = criterion(test_outputs, y_test_sample)

        train_losses.append(loss.item())
        test_losses.append(test_loss.item())

        print(f"[TRAINER] Epoch {epoch+1}/{mini_epochs}, Train Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}")

    # Simulate processing time
    time.sleep(TRAINER_WAIT_TIME)

    # Update indices and counters
    out_index_trainer = (out_index_trainer + TRAINER_READ_THRESHOLD) % CAPACITY
    read_datapoints_in_trainer += TRAINER_READ_THRESHOLD

training_end_time = time.time()
print(f"[TRAINER] Completed in {training_end_time - training_start_time:.2f} seconds")

inference_start_time = time.time()

# Initialize inference
inference = Inference()
inference.set_model(model)

# Process test data one by one
read_datapoints_in_inferer = 0
out_index_inferer = 0
all_predictions = []

while read_datapoints_in_inferer < len(X_test):
    if read_datapoints_in_inferer >= len(X_test):
        break

    # Get one test example
    test_sample = X_test[read_datapoints_in_inferer].unsqueeze(0)  # Add batch dimension

    # Make prediction
    prediction = inference.predict(test_sample)[0]  # Get the first (and only) prediction
    all_predictions.append(prediction)

    # Simulate processing time
    time.sleep(INFERENCE_WAIT_TIME)

    # Update indices and counters
    read_datapoints_in_inferer += INFERENCE_READ_THRESHOLD

    # Only process a subset of the test data for demonstration
    if read_datapoints_in_inferer >= CAPACITY:
        break

inference_end_time = time.time()
print(f"[INFERENCE] Completed in {inference_end_time - inference_start_time:.2f} seconds")

analysis_start_time = time.time()

# Convert predictions to numpy array
predictions = np.array(all_predictions)

# Analyze model performance
analyzer = Analysis()
analyzer.set_data(y_test[:len(predictions)], predictions)
metrics = analyzer.calculate_metrics()
errors, error_stats = analyzer.analyze_error_distribution()

analysis_end_time = time.time()
print(f"[ANALYSIS] Completed in {analysis_end_time - analysis_start_time:.2f} seconds")

display_start_time = time.time()

# Visualize results
displayer = Display()
original_data = preprocessor.get_original_data()
displayer.set_data(original_data, predictions, start_idx=-len(predictions))

# Plot results
displayer.plot_results(title='LSTM Temperature Forecast')

# Plot by year
displayer.plot_by_year()

# Plot error histogram
displayer.plot_error_histogram(errors)

# Plot training history
displayer.plot_training_history(train_losses, test_losses)

display_end_time = time.time()
print(f"[DISPLAY] Completed in {display_end_time - display_start_time:.2f} seconds")

main_end_time = time.time()
print(f'[CONSOLE] Total operation time taken is {main_end_time - main_start_time:.2f} seconds')

# Print summary of metrics
print("\nModel Performance Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

[DATA LOADER] Added batch 1 to buffer position 1, size: 813
[DATA LOADER] Added batch 2 to buffer position 2, size: 813
[DATA LOADER] Added batch 3 to buffer position 3, size: 813
[DATA LOADER] Added batch 4 to buffer position 4, size: 813
[DATA LOADER] Added batch 5 to buffer position 5, size: 813
[DATA LOADER] Added batch 6 to buffer position 6, size: 813
[DATA LOADER] Added batch 7 to buffer position 7, size: 813
[DATA LOADER] Added batch 8 to buffer position 8, size: 813
[DATA LOADER] Added batch 9 to buffer position 9, size: 813
[DATA LOADER] Added batch 10 to buffer position 10, size: 813
[DATA LOADER] Added batch 11 to buffer position 11, size: 813
[DATA LOADER] Added batch 12 to buffer position 12, size: 813
[DATA LOADER] Added batch 13 to buffer position 13, size: 813
[DATA LOADER] Added batch 14 to buffer position 14, size: 813
[DATA LOADER] Added batch 15 to buffer position 15, size: 813
[DATA LOADER] Completed in 15.17 seconds
[TRAINER] Processing batches from positions 1 t

RuntimeError: Placeholder storage has not been allocated on MPS device!